In [1]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [9]:
query = f"""

select 
    distinct
    referreruserid parent_id,
    referreruserid,
    referreduserid id,
    referreduserid
from experiments.customer_referral_fraud_22
limit 1000
"""

In [10]:
df = pd.read_sql(query, conn)
df

,parent_id,referreruserid,id,referreduserid
0,62a41cb117af9ac29753afd3,62a41cb117af9ac29753afd3,635fc1de85b399e43b563c6a,635fc1de85b399e43b563c6a
1,635fc095ae48a77cd3d3958f,635fc095ae48a77cd3d3958f,635fc1ec6cb836180fa918ba,635fc1ec6cb836180fa918ba
2,61d217f9a7574677b2e620ac,61d217f9a7574677b2e620ac,635fc164a63f416a29722389,635fc164a63f416a29722389
3,628b5204cf1a867f2d120eee,628b5204cf1a867f2d120eee,635fc173e278cb85cc693970,635fc173e278cb85cc693970
4,59c5d0dd61ccc84d5c3c4108,59c5d0dd61ccc84d5c3c4108,635fc01185b399d08c563b15,635fc01185b399d08c563b15
...,...,...,...,...
995,620c754a57d2b22013b0b038,620c754a57d2b22013b0b038,635f5f75c6d7c23f20157ebc,635f5f75c6d7c23f20157ebc
996,6162a03d37f15f2aba9116ac,6162a03d37f15f2aba9116ac,635f6019c6d7c28276157f52,635f6019c6d7c28276157f52
997,626cd6a57975624cab455576,626cd6a57975624cab455576,635f5eac85b39925bd55ef9f,635f5eac85b39925bd55ef9f
998,62614e8b7baa9a86c24ae9f4,62614e8b7baa9a86c24ae9f4,635f5e7eb087cd165a1dd36b,635f5e7eb087cd165a1dd36b


In [11]:
def count_descendants(df, parent_id):
    total = 0
    children = df[df['parent_id'] == parent_id]['id']
    for child in children:
        total += 1 + count_descendants(df, child)
    return total

In [12]:
ancestor_id = '62c920a53159e3064b05383b'
descendants_count = count_descendants(df, ancestor_id)

In [13]:
descendants_count

0

In [14]:
from collections import defaultdict

In [15]:
def get_referred(referrer, table, referrer_to_referred):
    referred_list = []
    for index, row in table.iterrows():
        if row['referreruserid'] == referrer:
            referred = row['referreduserid']
            if referred not in referrer_to_referred:
                referred_list.append(referred)
                referrer_to_referred[referred] = get_referred(referred, table, referrer_to_referred)
    return referred_list

referrer_to_referred = defaultdict(list)

for index, row in df.iterrows():
    referrer = row['referreruserid']
    if referrer not in referrer_to_referred:
        referrer_to_referred[referrer] = get_referred(referrer, df, referrer_to_referred)

for referrer, referred_list in referrer_to_referred.items():
    print(f"family: {referrer} {referred_list}")

family: 635fc1de85b399e43b563c6a []
family: 62a41cb117af9ac29753afd3 ['635fc1de85b399e43b563c6a']
family: 635fc29ec6d7c20caa15caf2 []
family: 635fc25a85b3990a15563cca ['635fc29ec6d7c20caa15caf2']
family: 635fc1ec6cb836180fa918ba ['635fc25a85b3990a15563cca']
family: 635fc095ae48a77cd3d3958f ['635fc1ec6cb836180fa918ba']
family: 635fc164a63f416a29722389 []
family: 61d217f9a7574677b2e620ac ['635fc164a63f416a29722389']
family: 635fc173e278cb85cc693970 []
family: 628b5204cf1a867f2d120eee ['635fc173e278cb85cc693970']
family: 635fc01185b399d08c563b15 []
family: 59c5d0dd61ccc84d5c3c4108 ['635fc01185b399d08c563b15']
family: 635f322d85b3993a9e55d050 []
family: 624d336ea060e849c0f389da ['635f322d85b3993a9e55d050']
family: 635fc299e278cb5dcc693a74 []
family: 6247091b317867366e645bf1 ['635fc299e278cb5dcc693a74']
family: 635fc2bc5ae34f7f5fd8b222 []
family: 635bec76a6f3c7d3c367d107 ['635fc2bc5ae34f7f5fd8b222']
family: 635fc2c3c6d7c2883015cb0c []
family: 61bcb6d617dd6a6d9116dac8 ['635fc2c3c6d7c2883015c